The main solving big ridge regression problems

In [1]:
workspace()

In [2]:
using Plots

In [109]:
using StatsBase
using JLD  
include("../src/BigRidge.jl")

plot_outputs_Plots (generic function with 1 method)

In [114]:
## Basic parameters
maxiter=2000;
max_time = 200;
max_epocs = 100;
printiters = true;
exacterror =true; 
tol = 10.0^(-5.0);
restol = 10.0^(-5.0);
skip_error_calculation = 25;   # number of iterations where error is not calculated (to save time!) ceil((m)^(1/2))

In [111]:
dataset = "rcv1_train.multiclass"   # news20.binary  rcv1_train.multiclass rcv1_train.binary mnist.scale  a9a  gisette_scale 
prob = defineProb(dataset);
showDetails(dataset)

../data/rcv1_train.multiclass.jld
ErrorException("File ../data/rcv1_train.multiclass-xsol.jld cannot be found")


Dict{Symbol,Any} with 3 entries:
  :sparsity => 0.00139868
  :dims     => 47236
  :n        => 15564

In [112]:
norm(prob.A*prob.xsol - prob.b)

1.3382982495227205e-10

In [113]:
options = MyOptions(0,"uni",tol,restol,maxiter,skip_error_calculation,max_time,printiters,exacterror,[]) 
OUTPUTS = [];

In [115]:
options.AUX = [0 2]; # don't use sign flipping and use a density of 10 per row
method_name = "rademacher";# ROS
options.sketchsize = floor((prob.n)^(1/2)); #
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
options.AUX = [0 5]; # don't use sign flipping and use a density of 10 per row
method_name = "rademacher";# ROS
options.sketchsize = 50; #floor((prob.n)^(1/2));
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
options.AUX = [0 10]; # don't use sign flipping and use a density of 10 per row
method_name = "rademacher";# ROS
options.sketchsize = 5; #floor((prob.n)^(1/2));
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
options.AUX = [0 50]; # don't use sign flipping and use a density of 10 per row
method_name = "rademacher";# ROS
options.sketchsize = 2; #floor((prob.n)^(1/2));
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

rademacher-217-2
-------------------
It   | Error% | Residual |  Time   
-------------------
  4  | 80.86  |  78.86  | 3.6481 
  8.044802 seconds (377.96 k allocations: 1.017 GB, 7.07% gc time)
rademacher-50-5
-------------------
It   | Error% | Residual |  Time   
-------------------
 10  | 75.67  |  58.89  | 3.8501 
  8.151849 seconds (15.51 k allocations: 1.080 GB, 1.26% gc time)
rademacher-5-10
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 88.63  |  83.86  | 1.2977 
 50  | 81.13  |  68.53  | 2.5685 
 57  | 80.18  |  67.30  | 3.0520 
 11.295594 seconds (12.43 k allocations: 1.114 GB, 0.87% gc time)
rademacher-2-50
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 89.50  |  84.04  | 2.0969 
 35  | 87.94  |  79.75  | 3.0272 
  9.340870 seconds (4.98 k allocations: 1.263 GB, 0.76% gc time)


In [98]:
# Testing Hadamard Sketch
# method_name = "Hadamard";# ROS
# options.sketchsize =10; # Int(floor(log2(prob.n)));
# options.skip_error_calculation =25;
# @time  output= solveRidge(prob, method_name, options);
# OUTPUTS = [OUTPUTS ; output];

In [99]:
# Testing countmin 
# options.sketchsize =10;#floor((prob.n)^(1/2));
# options.skip_error_calculation =25;
# method_name = "countmin";# gradient descent
# @time output= solveRidge(prob, method_name, options);
# OUTPUTS = [OUTPUTS ; output];

In [101]:
# Testing CG 
method_name = "CG";# gradient descent
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CG
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 1.21  |  1.22  | 0.0042 
 50  | 0.41  |  0.46  | 0.0084 
 75  | 0.25  |  0.36  | 0.0123 
100  | 0.18  |  0.17  | 0.0160 
125  | 0.15  |  0.18  | 0.0199 
150  | 0.12  |  0.13  | 0.0287 
175  | 0.11  |  0.10  | 0.0321 
200  | 0.10  |  0.09  | 0.0364 
225  | 0.09  |  0.07  | 0.0397 
250  | 0.08  |  0.07  | 0.0429 
275  | 0.07  |  0.05  | 0.0460 
300  | 0.07  |  0.10  | 0.0496 
325  | 0.06  |  0.05  | 0.0531 
350  | 0.06  |  0.06  | 0.0561 
375  | 0.05  |  0.10  | 0.0730 
400  | 0.05  |  0.08  | 0.0837 
425  | 0.05  |  0.05  | 0.0866 
450  | 0.04  |  0.03  | 0.0918 
475  | 0.04  |  0.06  | 0.0947 
500  | 0.04  |  0.05  | 0.1132 
525  | 0.04  |  0.04  | 0.1155 
550  | 0.03  |  0.06  | 0.1181 
575  | 0.03  |  0.03  | 0.1301 
600  | 0.03  |  0.03  | 0.1324 
625  | 0.03  |  0.14  | 0.1360 
650  | 0.03  |  0.05  | 0.1383 
675  | 0.02  |  0.04  | 0.1405 
700  | 0.02  |  0.04  | 0.1427 
725  | 0.02  |  0.02  | 0

In [104]:
# Testing CD 
options.maxiter=5000;
options.skip_error_calculation = 50;
method_name = "CD";# gradient descent
options.sketchsize = 10;#floor((prob.n)^(1/2));
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
method_name = "CD";# gradient descent
options.sketchsize = 50;
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CD-10
-------------------
It   | Error% | Residual |  Time   
-------------------
 50  | 15.83  |  10.91  | 0.0060 
100  | 8.60  |  4.83  | 0.0107 
150  | 6.18  |  3.58  | 0.0152 
200  | 5.06  |  2.86  | 0.0194 
250  | 4.10  |  2.78  | 0.0235 
300  | 3.06  |  2.42  | 0.0357 
350  | 2.86  |  2.27  | 0.0394 
400  | 2.33  |  1.64  | 0.0436 
450  | 2.03  |  1.65  | 0.0476 
500  | 1.79  |  1.36  | 0.0510 
550  | 1.57  |  1.27  | 0.0542 
600  | 1.36  |  1.23  | 0.0573 
650  | 1.24  |  1.17  | 0.0642 
700  | 1.11  |  1.07  | 0.0671 
750  | 1.02  |  0.93  | 0.0699 
800  | 0.93  |  0.90  | 0.0727 
850  | 0.83  |  1.00  | 0.0754 
900  | 0.77  |  0.86  | 0.0782 
950  | 0.73  |  0.72  | 0.0809 
1000  | 0.66  |  0.74  | 0.0874 
1050  | 0.61  |  0.70  | 0.0900 
1100  | 0.55  |  0.68  | 0.0926 
1150  | 0.50  |  0.60  | 0.0952 
1200  | 0.46  |  0.60  | 0.0978 
1250  | 0.42  |  0.63  | 0.1004 
1300  | 0.51  |  0.75  | 0.1062 
1350  | 0.36  |  0.62  | 0.1089 
1400  | 0.34  |  0.50  | 0.1120 
1450  | 0.3

In [87]:
# OUTPUTS =[];
# include("../src/call_all_methods.jl")

In [105]:
savename = replace(prob.name, r"[\/]", "-");
save("../data/$(savename)-OUTPUTS.jld", "OUTPUTS", OUTPUTS)

In [106]:
#using GR
using Plots
pgfplots()
plot_outputs_Plots(OUTPUTS,savename)

mnist.scale


In [344]:
# Loading saved problem data
probname = "gisette_scale"#"gisette_scale";  
test = load("../data/$(probname)-OUTPUTS.jld")
OUTPUTS = test["OUTPUTS"];

In [108]:
# a9a
OUTPUTS = []
options.tol = 10.0^(-9.0);
options.restol = 10.0^(-9.0);
options.AUX = [0 2] # don't use sign flipping and use a density of 10 per row
method_name = "rademacher";# ROS
options.sketchsize = floor((prob.n)^(1/2)); #
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
method_name = "CG";# gradient descent
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
options.maxiter=5000;
options.skip_error_calculation = 100;
method_name = "CD";# gradient descent
options.sketchsize = 10;#floor((prob.n)^(1/2));
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
plot_outputs_Plots(OUTPUTS,savename)

rademacher-27-2
-------------------
It   | Error% | Residual |  Time   
-------------------
100  | 2.01  |  1.63  | 0.1510 
200  | 0.84  |  0.70  | 0.2824 
300  | 0.36  |  0.43  | 0.4112 
400  | 0.50  |  0.26  | 0.5403 
500  | 0.64  |  0.19  | 0.6747 
600  | 0.07  |  0.15  | 0.7942 
700  | 0.05  |  0.12  | 0.9164 
800  | 0.02  |  0.08  | 1.0451 
900  | 0.01  |  0.06  | 1.1706 
1000  | 0.02  |  0.04  | 1.2876 
1100  | 0.03  |  0.04  | 1.4109 
1200  | 0.05  |  0.03  | 1.5242 
1300  | 0.06  |  0.02  | 1.6328 
1400  | 0.07  |  0.02  | 1.7407 
1500  | 0.08  |  0.01  | 1.8531 
1600  | 0.09  |  0.01  | 1.9677 
1700  | 0.07  |  0.01  | 2.0762 
1800  | 0.01  |  0.01  | 2.1934 
1900  | 0.02  |  0.01  | 2.2916 
2000  | 0.00  |  0.01  | 2.4050 
2100  | 0.00  |  0.00  | 2.5393 
2200  | 0.00  |  0.00  | 2.7023 
2300  | 0.00  |  0.00  | 2.9232 
2361  | 0.00  |  0.00  | 3.0002 
  3.077364 seconds (1.73 M allocations: 1.277 GB, 6.27% gc time)
CG
-------------------
It   | Error% | Residual |  Time   
-